In [2]:
import spacy

In [6]:
nlp = spacy.load("output/model-best")

In [7]:
text = "data kontrak tenor yg lebih dari 1 tahun"

doc = nlp(text)

[(w.text, w.label_) for w in doc.ents]

[('kontrak', 'SUBJECT'),
 ('tenor', 'ATTRIBUTE'),
 ('lebih', 'LOGIC'),
 ('1 tahun', 'VALUE')]

Five labels for now: subject, attribute, logic, value and location (same place as value, will instantly make attribute = location)

```sql
SELECT * FROM (subject) WHERE (attribute) (logic) '(value)'
```

In [8]:
subject = ""; attribute = ""; logic = ""; value = ""; location = ""

for w in doc.ents:
    if w.label_ == "SUBJECT":
        subject = w.text
    if w.label_ == "ATTRIBUTE":
        attribute = w.text
    if w.label_ == "LOGIC":
        logic = w.text
    if w.label_ == "VALUE" or w.label_ == "LOCATION":
        value = w.text
    if w.label_ == "LOCATION":
        location = w.text

if location != "":
    attribute = "location"

if logic == "":
    logic = "="
elif logic == "kurang" or logic == "bawah":
    logic = "<"
elif logic == "lebih" or logic == "atas":
    logic = ">"

if attribute[-3:] == "nya":
    attribute = attribute[:-3]

sql = f"SELECT * FROM {subject} WHERE {attribute} {logic} '{value}'"
print(sql)

SELECT * FROM kontrak WHERE tenor > '1 tahun'
